In [1]:
import pygame
from collections import deque

# Constants
WIDTH, HEIGHT = 400, 400
ROWS, COLS = 10, 10
CELL_SIZE = WIDTH // COLS

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (160, 160, 160)
YELLOW = (255, 255, 0)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

# Maze layout (0 = path, 1 = wall)
MAZE = [
    [0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 1, 0, 1, 0, 1, 0, 1, 1, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
    [1, 1, 0, 1, 1, 1, 1, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 1, 1, 1, 1, 0, 1, 1, 1, 0],
    [0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 0, 1, 1, 1, 0],
]

GHOST_START = (0, 0)
PACMAN_START = (9, 9)

def bfs_path(grid, start, goal):
    rows, cols = len(grid), len(grid[0])
    visited = [[False]*cols for _ in range(rows)]
    prev = [[None]*cols for _ in range(rows)]
    queue = deque([start])
    visited[start[0]][start[1]] = True
    directions = [(-1,0),(1,0),(0,-1),(0,1)]

    while queue:
        r, c = queue.popleft()
        if (r, c) == goal:
            break
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            if 0 <= nr < rows and 0 <= nc < cols and not visited[nr][nc] and grid[nr][nc] == 0:
                queue.append((nr, nc))
                visited[nr][nc] = True
                prev[nr][nc] = (r, c)

    path = []
    cur = goal
    while cur and cur != start:
        path.append(cur)
        cur = prev[cur[0]][cur[1]]
    path.reverse()
    return path

def draw_grid(win):
    for r in range(ROWS):
        for c in range(COLS):
            rect = pygame.Rect(c*CELL_SIZE, r*CELL_SIZE, CELL_SIZE, CELL_SIZE)
            color = GRAY if MAZE[r][c] == 1 else WHITE
            pygame.draw.rect(win, color, rect)
            pygame.draw.rect(win, BLACK, rect, 1)

def draw_path(win, path):
    for r, c in path:
        rect = pygame.Rect(c*CELL_SIZE, r*CELL_SIZE, CELL_SIZE, CELL_SIZE)
        pygame.draw.rect(win, GREEN, rect)

def draw_characters(win, ghost, pacman):
    gr, gc = ghost
    pr, pc = pacman
    pygame.draw.circle(win, BLUE, (gc*CELL_SIZE + CELL_SIZE//2, gr*CELL_SIZE + CELL_SIZE//2), CELL_SIZE//3)
    pygame.draw.circle(win, YELLOW, (pc*CELL_SIZE + CELL_SIZE//2, pr*CELL_SIZE + CELL_SIZE//2), CELL_SIZE//3)

def draw_score_and_status(win, moves, game_over):
    font = pygame.font.SysFont(None, 24)
    score_text = font.render(f"Moves: {moves}", True, RED)
    win.blit(score_text, (10, 10))

    if game_over:
        over_font = pygame.font.SysFont(None, 36)
        over_text = over_font.render("Pac-Man was caught!", True, RED)
        win.blit(over_text, (WIDTH // 2 - 120, HEIGHT // 2 - 20))

def main():
    pygame.init()
    win = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Pac-Man BFS Pathfinding")

    ghost = GHOST_START
    pacman = PACMAN_START
    clock = pygame.time.Clock()
    running = True
    move_count = 0
    game_over = False

    ghost_timer = 0
    GHOST_DELAY = 5  

    while running:
        clock.tick(10)  

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        if not game_over:
            # Pac-Man controls
            keys = pygame.key.get_pressed()
            pr, pc = pacman
            if keys[pygame.K_UP] and pr > 0 and MAZE[pr-1][pc] == 0:
                pr -= 1
            elif keys[pygame.K_DOWN] and pr < ROWS-1 and MAZE[pr+1][pc] == 0:
                pr += 1
            elif keys[pygame.K_LEFT] and pc > 0 and MAZE[pr][pc-1] == 0:
                pc -= 1
            elif keys[pygame.K_RIGHT] and pc < COLS-1 and MAZE[pr][pc+1] == 0:
                pc += 1
            pacman = (pr, pc)

            # Ghost movement
            ghost_timer += 1
            if ghost_timer >= GHOST_DELAY:
                ghost_timer = 0
                path = bfs_path(MAZE, ghost, pacman)
                if path:
                    ghost = path[0]
                    move_count += 1

            if ghost == pacman:
                game_over = True

        win.fill(BLACK)
        draw_grid(win)
        if not game_over:
            path = bfs_path(MAZE, ghost, pacman)
            draw_path(win, path)
        draw_characters(win, ghost, pacman)
        draw_score_and_status(win, move_count, game_over)
        pygame.display.update()

    pygame.quit()

if __name__ == "__main__":
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
import pygame
import random
from collections import deque

# Grid and window setup
ROWS, COLS = 20, 20
CELL_SIZE = 20
WIDTH, HEIGHT = COLS * CELL_SIZE, ROWS * CELL_SIZE

# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 200, 0)
RED = (200, 0, 0)
BLUE = (0, 100, 255)

# Directions
DIRS = [(-1,0),(1,0),(0,-1),(0,1)]

def bfs(grid, start, goal):
    queue = deque([start])
    visited = {start: None}
    while queue:
        current = queue.popleft()
        if current == goal:
            break
        for dr, dc in DIRS:
            nr, nc = current[0] + dr, current[1] + dc
            if 0 <= nr < ROWS and 0 <= nc < COLS and grid[nr][nc] == 0 and (nr, nc) not in visited:
                queue.append((nr, nc))
                visited[(nr, nc)] = current
    if goal not in visited:
        return []
    # Reconstruct path
    path = []
    cur = goal
    while cur != start:
        path.append(cur)
        cur = visited[cur]
    path.reverse()
    return path

def random_food(snake):
    while True:
        r, c = random.randint(0, ROWS - 1), random.randint(0, COLS - 1)
        if (r, c) not in snake:
            return (r, c)

def draw(win, snake, food, path, score):
    win.fill(BLACK)

    # Draw grid
    for r in range(ROWS):
        for c in range(COLS):
            pygame.draw.rect(win, WHITE, (c * CELL_SIZE, r * CELL_SIZE, CELL_SIZE, CELL_SIZE), 1)

    # Draw path
    for r, c in path:
        pygame.draw.rect(win, BLUE, (c * CELL_SIZE, r * CELL_SIZE, CELL_SIZE, CELL_SIZE))

    # Draw snake
    for r, c in snake:
        pygame.draw.rect(win, GREEN, (c * CELL_SIZE, r * CELL_SIZE, CELL_SIZE, CELL_SIZE))

    # Draw food
    fr, fc = food
    pygame.draw.rect(win, RED, (fc * CELL_SIZE, fr * CELL_SIZE, CELL_SIZE, CELL_SIZE))

    # Draw score
    font = pygame.font.SysFont(None, 24)
    score_text = font.render(f"Score: {score} | Length: {len(snake)}", True, WHITE)
    win.blit(score_text, (10, 5))

    pygame.display.update()

def main():
    pygame.init()
    win = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Snake AI with BFS")
    clock = pygame.time.Clock()

    snake = [(10, 10)]
    food = random_food(snake)
    grid = [[0]*COLS for _ in range(ROWS)]
    score = 0
    running = True

    while running:
        clock.tick(20)

       
        grid = [[0]*COLS for _ in range(ROWS)]
        for r, c in snake:
            grid[r][c] = 1

        path = bfs(grid, snake[0], food)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        if not path:
            print("❌ No path to food. Game Over.")
            break

        next_cell = path[0]

       
        if (
            next_cell[0] < 0 or next_cell[0] >= ROWS or
            next_cell[1] < 0 or next_cell[1] >= COLS or
            next_cell in snake
        ):
            print("💥 Collision! Game Over.")
            break

        snake.insert(0, next_cell)

        if next_cell == food:
            score += 5
            food = random_food(snake)
        else:
            snake.pop()

        draw(win, snake, food, path, score)

    pygame.quit()

if __name__ == "__main__":
    main()


❌ No path to food. Game Over.
